In [1]:
import random as rd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error #, r2_score
from sklearn.model_selection import train_test_split

import gspread
from sqlalchemy import create_engine
import re

gc = gspread.service_account(filename="./credentials.json")

import environ
env = environ.Env()
database_url = env.str(
    var="DATABASE_URL", default="postgres://postgres:@postgres:5432/postgres"
)
engine = create_engine(database_url, echo=False)

In [41]:
fb = pd.read_sql("etl_marketing_facebook", engine)
fb = fb[fb['type'] == 'daily']

In [42]:
fb.head()

,ad_id,ad_name,adset_name,campaign_id,campaign_name,clicks,cpc,cpm,cpp,ctr,...,unique_clicks,campaign_objective,status,type,landing_page_view,product,objective,Campaign name,artwork,Content
0,23845924248770634,"Post: ""เคยเจอปัญหาต้องเข้าออฟฟิศ เพื่อเซ็นเอกส...","Post: ""เคยเจอปัญหาต้องเข้าออฟฟิศ เพื่อเซ็นเอกส...",23845924248710634,PRO:EMEM_OBJ:ENGA_CN:NORM_AW:MULT_CONT:BOARD,65,1.588462,176.797945,202.848723,11.130137,...,65,POST_ENGAGEMENT,PAUSED,daily,None,EMEM,ENGA,NORM,MULT,BOARD
1,23845924248770634,"Post: ""เคยเจอปัญหาต้องเข้าออฟฟิศ เพื่อเซ็นเอกส...","Post: ""เคยเจอปัญหาต้องเข้าออฟฟิศ เพื่อเซ็นเอกส...",23845924248710634,PRO:EMEM_OBJ:ENGA_CN:NORM_AW:MULT_CONT:BOARD,74,2.211486,168.01848,233.452211,7.597536,...,72,POST_ENGAGEMENT,PAUSED,daily,None,EMEM,ENGA,NORM,MULT,BOARD
2,23845924248770634,"Post: ""เคยเจอปัญหาต้องเข้าออฟฟิศ เพื่อเซ็นเอกส...","Post: ""เคยเจอปัญหาต้องเข้าออฟฟิศ เพื่อเซ็นเอกส...",23845924248710634,PRO:EMEM_OBJ:ENGA_CN:NORM_AW:MULT_CONT:BOARD,79,2.166329,135.610143,196.93901,6.259905,...,78,POST_ENGAGEMENT,PAUSED,daily,None,EMEM,ENGA,NORM,MULT,BOARD
3,23845924248770634,"Post: ""เคยเจอปัญหาต้องเข้าออฟฟิศ เพื่อเซ็นเอกส...","Post: ""เคยเจอปัญหาต้องเข้าออฟฟิศ เพื่อเซ็นเอกส...",23845924248710634,PRO:EMEM_OBJ:ENGA_CN:NORM_AW:MULT_CONT:BOARD,21,2.950476,152.610837,179.594203,5.172414,...,21,POST_ENGAGEMENT,PAUSED,daily,None,EMEM,ENGA,NORM,MULT,BOARD
4,23845970223610634,Video-071020,Biz-071020,23845806592580634,PRO:EMEM_OBJ:LEAD_CN:NORM_AW:SING_CONT:MIX,15,4.051333,394.61039,407.852349,9.74026,...,14,LEAD_GENERATION,PAUSED,daily,None,EMEM,LEAD,NORM,SING,MIX


In [43]:
fb.dtypes

ad_id                 object
ad_name               object
adset_name            object
campaign_id           object
campaign_name         object
clicks                object
cpc                   object
cpm                   object
cpp                   object
ctr                   object
date_start            object
date_stop             object
frequency             object
impressions           object
post_engagement       object
reach                 object
spend                 object
unique_clicks         object
campaign_objective    object
status                object
type                  object
landing_page_view     object
product               object
objective             object
Campaign name         object
artwork               object
Content               object
dtype: object

In [44]:
fb_ads = fb[['ad_name']]
fb_ads

,ad_name
0,"Post: ""เคยเจอปัญหาต้องเข้าออฟฟิศ เพื่อเซ็นเอกส..."
1,"Post: ""เคยเจอปัญหาต้องเข้าออฟฟิศ เพื่อเซ็นเอกส..."
2,"Post: ""เคยเจอปัญหาต้องเข้าออฟฟิศ เพื่อเซ็นเอกส..."
3,"Post: ""เคยเจอปัญหาต้องเข้าออฟฟิศ เพื่อเซ็นเอกส..."
4,Video-071020
...,...
397,AW:SING_CONT:WHYCDM
398,AW:SING_CONT:4STEPS_
399,AW:VIDEO_CONT:FEATURES
400,AW:SING_CONT:BOSS-VS-STAFF


In [45]:
data_count = fb_ads.groupby('ad_name').size().reset_index(name='counts')

In [46]:
data_count

,ad_name,counts
0,3C - Multiple,2
1,3C-Multiple,1
2,AW:MULT_CONT:WFH-3PRODUCTS,18
3,AW:SING_CONT:3BENEFITS,5
4,AW:SING_CONT:4STEPS_,115
5,AW:SING_CONT:4STEPS_ - Copy,6
6,AW:SING_CONT:BOSS-VS-STAFF,4
7,AW:SING_CONT:COVID,6
8,AW:SING_CONT:FILES,5
9,AW:SING_CONT:SENIOR-DEV,8


In [47]:
fb['date'] = pd.to_datetime(fb['date_start'])

fb['week'] = fb['date'].dt.isocalendar().week
fb['month'] = fb['date'].dt.month
fb['year'] = fb['date'].dt.year
fb_data = fb[['ad_name', 'week', 'month', 'year']]

fb_data

,ad_name,week,month,year
0,"Post: ""เคยเจอปัญหาต้องเข้าออฟฟิศ เพื่อเซ็นเอกส...",40,10,2020
1,"Post: ""เคยเจอปัญหาต้องเข้าออฟฟิศ เพื่อเซ็นเอกส...",40,10,2020
2,"Post: ""เคยเจอปัญหาต้องเข้าออฟฟิศ เพื่อเซ็นเอกส...",40,10,2020
3,"Post: ""เคยเจอปัญหาต้องเข้าออฟฟิศ เพื่อเซ็นเอกส...",41,10,2020
4,Video-071020,41,10,2020
...,...,...,...,...
397,AW:SING_CONT:WHYCDM,9,3,2021
398,AW:SING_CONT:4STEPS_,9,3,2021
399,AW:VIDEO_CONT:FEATURES,9,3,2021
400,AW:SING_CONT:BOSS-VS-STAFF,9,3,2021


In [52]:
data_count_1 = fb_data.groupby(['year','month', 'week']).agg(['count'])

In [53]:
data_count_1

ad_name
                  count
year month week        
2020 10    40         3
           41        10
           42        19
           43         9
           44         5
     11    45         6
           46        10
           47         7
           48         9
           49         2
     12    49        16
           50        14
           51        11
           52        24
           53         4
2021 1     1         35
           2         44
           3         27
           4         23
     2     5         30
           6         22
           7         27
           8         26
     3     9         19

In [54]:
gg = pd.read_sql("etl_google_ads", engine)

In [59]:
gg.head(2)

,Date,campaignname,id,adgroupname,criteria,status,clicks,impressions,Cost,ctr,conversions,conversionrate,costperconversion
96,2021-01-04,e-Memo,297102057024,Product 040121,ERP system,enabled,1.0,7.0,78.46,14.29%,0,0.00%,0
97,2021-01-05,e-Memo,297102057024,Product 040121,ERP system,enabled,2.0,24.0,63.68,8.33%,0,0.00%,0


In [57]:
gg['Cost'] = gg['Cost'].astype(float)
gg['clicks'] = gg['clicks'].astype(float)
gg['impressions'] = gg['impressions'].astype(float)
gg = gg.query('Cost != 0 and Cost < 200 and impressions <= 250 and clicks > 0')
gg

,Date,campaignname,id,adgroupname,criteria,status,clicks,impressions,Cost,ctr,conversions,conversionrate,costperconversion
96,2021-01-04,e-Memo,297102057024,Product 040121,ERP system,enabled,1.0,7.0,78.46,14.29%,0,0.00%,0
97,2021-01-05,e-Memo,297102057024,Product 040121,ERP system,enabled,2.0,24.0,63.68,8.33%,0,0.00%,0
98,2021-01-06,e-Memo,297102057024,Product 040121,ERP system,enabled,1.0,7.0,6.64,14.29%,0,0.00%,0
104,2021-01-21,e-Memo,297102057024,Product 040121,ERP system,enabled,1.0,2.0,22.91,50.00%,0,0.00%,0
274,2021-01-12,e-Memo,297102057024,Product 040121,ERP system,enabled,1.0,12.0,13.11,8.33%,0,0.00%,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154947,2021-03-02,etax,1235025025847,Product 160221,ใบเสร็จ รับ เงิน กับ ใบ กํา กับ ภาษี,enabled,3.0,49.0,28.79,6.12%,1,33.33%,28.79
154948,2021-03-03,etax,1235025025847,Product 160221,ใบเสร็จ รับ เงิน กับ ใบ กํา กับ ภาษี,enabled,5.0,56.0,57.54,8.93%,0,0.00%,0
155002,2021-02-19,etax,1235025025887,Product 160221,ใบเสร็จ ใบกํากับภาษี,enabled,2.0,3.0,25.70,66.67%,0,0.00%,0
155006,2021-03-02,etax,1235025025887,Product 160221,ใบเสร็จ ใบกํากับภาษี,enabled,6.0,49.0,68.97,12.24%,0,0.00%,0


In [58]:
gg.dtypes

Date                  object
campaignname          object
id                    object
adgroupname           object
criteria              object
status                object
clicks               float64
impressions          float64
Cost                 float64
ctr                   object
conversions           object
conversionrate        object
costperconversion     object
dtype: object

In [65]:
gg['Date'] = pd.to_datetime(gg['Date'])

gg['week'] = gg['Date'].dt.isocalendar().week
gg['month'] = gg['Date'].dt.month
gg['year'] = gg['Date'].dt.year
gg.rename(columns={'criteria':'keywords'}, inplace=True)
data_gg = gg[['keywords', 'year', 'month', 'week']]
data_gg

<ipython-input-65-7c58235fe34e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gg['Date'] = pd.to_datetime(gg['Date'])
<ipython-input-65-7c58235fe34e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gg['week'] = gg['Date'].dt.isocalendar().week
<ipython-input-65-7c58235fe34e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,keywords,year,month,week
96,ERP system,2021,1,1
97,ERP system,2021,1,1
98,ERP system,2021,1,1
104,ERP system,2021,1,3
274,ERP system,2021,1,2
...,...,...,...,...
154947,ใบเสร็จ รับ เงิน กับ ใบ กํา กับ ภาษี,2021,3,9
154948,ใบเสร็จ รับ เงิน กับ ใบ กํา กับ ภาษี,2021,3,9
155002,ใบเสร็จ ใบกํากับภาษี,2021,2,7
155006,ใบเสร็จ ใบกํากับภาษี,2021,3,9


In [79]:
data_gg[data_gg['week']==1]['keywords']

96                        ERP system
97                        ERP system
98                        ERP system
4371        electronic signature pdf
5015       content management system
10252            เซ็น เอกสาร ออนไลน์
18288            customer management
35116            e digital signature
35266                      paperless
35375                   Thailand 4.0
35964                     crm system
36461                     sign a pdf
49360                    ระบบ เอกสาร
53449     ระบบ เอกสาร อิเล็กทรอนิกส์
56258              การ จัดการ เอกสาร
56260              การ จัดการ เอกสาร
61324                    ระบบ เอกสาร
61325                    ระบบ เอกสาร
70771             เซ็น เอกสารออนไลน์
74919              workflow programs
79707                 crm management
83898                    e-signature
96614                 salesforce crm
97717                   ทำงานที่บ้าน
99722                  จัดการ เอกสาร
103621             virtual signature
121536           digital e signature
1

In [66]:
data_count_2 = data_gg.groupby(['year','month', 'week']).agg(['count'])

In [67]:
data_count_2

keywords
                   count
year month week         
2020 11    45         26
           46         39
           47         19
           48         38
           49          5
     12    49         25
           51         24
           52         90
2021 1     1          34
           2          99
           3          84
           4          80
     2     5          67
           6         101
           7         103
           8         123
     3     9         108

In [73]:
# data_gg['keywords'].unique()
for colm in ['keywords']:
    print(colm)

    colm_vals = data_gg[colm].unique()
    for i, k in enumerate(colm_vals):
        print('{}   | {} ' . format(i,k))  
    print('-----------------------')

keywords
0   | ERP system 
1   | tax invoice 
2   | electronic signature pdf 
3   | content management system 
4   | erp 
5   | digital document management solutions 
6   | document sign 
7   | workflow software 
8   | เซ็น เอกสาร ออนไลน์ 
9   | document signature 
10   | crm 
11   | approval system 
12   | โปรแกรม ธุรกิจ 
13   | ซอฟต์แวร์ การ จัดการ ธุรกิจ 
14   | crm software 
15   | +เซ็นเอกสาร +ออนไลน์ 
16   | ระบบ เอกสาร ออนไลน์ 
17   | customer relationship management 
18   | customer management 
19   | work from 
20   | digital certificate 
21   | electronic document management system 
22   | digital signature 
23   | electronic signature 
24   | ระบบเอกสารออนไลน์ 
25   | โปรแกรม จัดเก็บเอกสาร 
26   | service provider 
27   | paperless 
28   | crm customer relationship management 
29   | +โปรแกรม +ออนไลน์ 
30   | Enterprise software 
31   | esign 
32   | online signatures 
33   | e-receipt 
34   | Thailand 4.0 
35   | e digital signature 
36   | sign a pdf 
37   | crm system 
38

In [168]:
import re

txt = "TheraininSpain2"
x = re.match(r"[a-zA-Z0-9_]", txt)
print(x)

<re.Match object; span=(0, 1), match='T'>


In [169]:
if not re.findall(r"\W", txt):
    print('asd')

asd


In [199]:
a = 'abcababcababcab'
aa = 'affedaaffed'
b = ''
work = ''
for i,k in enumerate(aa):
    b += k
    c = re.findall(b, a)
    if len(c) >= 3:
        work = b
print(work)

        

a


In [206]:
def MathChallenge(strParam):

  # code goes here
    word = ''
    final_work = ''
    for i,k in enumerate(strParam):
        word += k
        c = re.findall(word, strParam)
        if len(c) >= 3:
            final_work = word
            print(c)

    return final_work if len(final_work) > 0 else -1

In [207]:
MathChallenge(a)

['a', 'a', 'a', 'a', 'a', 'a']
['ab', 'ab', 'ab', 'ab', 'ab', 'ab']
['abc', 'abc', 'abc']
['abca', 'abca', 'abca']
['abcab', 'abcab', 'abcab']


'abcab'

In [208]:
MathChallenge(aa)

['a', 'a', 'a']


'a'

In [239]:
def ArrayChallenge(arr):
    count = len(arr)
    sum_values = []

    # code goes here
    for i in range(count):
        for k in range(count - i):
            k = k + i +1
            sum_values.append(sum(arr[i:k]))

    return max(sum_values)

In [240]:
ArrayChallenge([1, -2, 0, 3])

3

In [241]:
ArrayChallenge([3, -1, -1, 4, 3, -1])

8

In [242]:
import requests
import numpy as np
import pandas as pd

r = requests.get('https://coderbyte.com/api/challenges/json/age-counting')
print(len(r.json()['data']))

5660


In [254]:
req = r.json()['data']

In [257]:
sp = req.split(',')

In [271]:
list_age = list()
for i in sp:
    if re.findall('age', i):
        if int(i[5:]) >= 50 :
            list_age.append(int(i[5:]))
print(len(list_age))

128


In [272]:
r.json()

{'data': 'key=IAfpK, age=58, key=WNVdi, age=64, key=jp9zt, age=47, key=0Sr4C, age=68, key=CGEqo, age=76, key=IxKVQ, age=79, key=eD221, age=29, key=XZbHV, age=32, key=k1SN5, age=88, key=4SCsU, age=65, key=q3kG6, age=33, key=MGQpf, age=13, key=Kj6xW, age=14, key=tg2VM, age=30, key=WSnCU, age=24, key=f1Vvz, age=46, key=dOS7A, age=72, key=tDojg, age=82, key=nZyJA, age=48, key=R8JTk, age=29, key=005Ot, age=66, key=HHROm, age=12, key=5yzG8, age=51, key=xMJ5D, age=38, key=TXtVu, age=82, key=Hz38B, age=84, key=WfObU, age=27, key=mmqYB, age=14, key=4Z3Ay, age=62, key=x3B0i, age=55, key=QCiQB, age=72, key=zGtmR, age=66, key=nlIN9, age=8, key=hKalB, age=50, key=Na33O, age=17, key=jMeXm, age=15, key=OO2Mc, age=32, key=hhowx, age=34, key=gLMJf, age=60, key=PblX6, age=66, key=8Vm5W, age=22, key=oZKd6, age=88, key=RXNfQ, age=25, key=3yy0p, age=64, key=FrQbL, age=80, key=vlUkk, age=55, key=DP8po, age=80, key=EroX6, age=84, key=3bsll, age=86, key=QhZjA, age=85, key=wm6uc, age=74, key=MC1FM, age=75, key

In [299]:
mkt = pd.read_sql("marketing_lead", engine)

In [289]:
mkt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   id                 298 non-null    int64              
 1   created_at         298 non-null    datetime64[ns, UTC]
 2   updated_at         298 non-null    datetime64[ns, UTC]
 3   source             298 non-null    object             
 4   compaign_name      298 non-null    object             
 5   ads_set_name       298 non-null    object             
 6   ads_id             298 non-null    object             
 7   name               298 non-null    object             
 8   company_name       298 non-null    object             
 9   phone              298 non-null    object             
 10  email              263 non-null    object             
 11  industry           298 non-null    object             
 12  status             298 non-null    object         

In [290]:
mkt_1 = mkt[(mkt['date'] >= "1/1/2021") & (mkt['date'] <= "31/1/2021")]

In [291]:
mkt_1 = mkt_1[mkt_1['status'] == 'WIN']

In [292]:
mkt_1

,id,created_at,updated_at,source,compaign_name,ads_set_name,ads_id,name,company_name,phone,...,datetime,company_size,lead_name,pipedrive_enable,pipedrive_is_send,day_of_week,date,time,details,value
9,198,2021-01-20 10:28:14.522182+00:00,2021-02-22 15:24:22.905731+00:00,FB ADS,PRO:EMEM_OBJ:CONV_AW:SING_CONT:4STEPS_,BISS-JOBSTITLE-TECH-060121,23846644567020634,Chaowalit Yongkunsiri,บจก. บีเอสแอล โกลบอล,925262591,...,2021-01-10 05:42:00+00:00,,Chaowalit Yongkunsiri: บจก. บีเอสแอล โกลบอล,False,False,6,2021-01-10,12:42:00,,0
16,234,2021-01-20 10:28:15.357089+00:00,2021-02-22 15:24:22.844946+00:00,MANUAL,,,,duangjai sittikan,เอสพีเค ซาวด์ ซิสเต็มส์,884967295,...,2021-01-18 03:33:00+00:00,,duangjai sittikan: เอสพีเค ซาวด์ ซิสเต็มส์,False,False,0,2021-01-18,10:33:00,,0
26,246,2021-01-20 10:28:16.672972+00:00,2021-02-22 15:24:22.861869+00:00,MANUAL,,,,,บ. ขอนเเก่นรื่นรมย์,,...,2021-01-03 17:00:00+00:00,,บ. ขอนเเก่นรื่นรมย์,False,False,0,2021-01-04,00:00:00,,0
42,232,2021-01-20 10:28:15.313056+00:00,2021-02-22 15:24:22.891958+00:00,FB ADS,PRO:EMEM_OBJ:CONV_AW:SING_CONT:4STEPS_,BISS-JOBSTITLE-TECH-060121,23846644567020634,khampee Puangpai,สหวิริยา สตีล อินดัสตรี จำกัด มหาชน,819159762,...,2021-01-16 16:30:00+00:00,,khampee Puangpai: สหวิริยา สตีล อินดัสตรี จำกั...,False,False,5,2021-01-16,23:30:00,,0
43,228,2021-01-20 10:28:15.225655+00:00,2021-02-22 15:24:22.894083+00:00,MANUAL,,,,,Habitat group,,...,2021-01-15 04:39:00+00:00,,Habitat group,False,False,4,2021-01-15,11:39:00,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,251,2021-01-20 10:28:16.774227+00:00,2021-02-22 15:24:23.388611+00:00,MANUAL,,,,ธนพล,medlifeplus,957509723,...,2021-01-09 17:00:00+00:00,,ธนพล: medlifeplus,False,False,6,2021-01-10,00:00:00,,0
287,255,2021-01-20 10:28:16.848185+00:00,2021-02-22 15:24:23.392516+00:00,FB ADS,,,,,บจก. สเเควร์ กรุ๊ป,,...,2021-01-10 17:00:00+00:00,,บจก. สเเควร์ กรุ๊ป,False,False,0,2021-01-11,00:00:00,,0
289,355,2021-01-30 06:45:41.080442+00:00,2021-02-22 15:24:23.396466+00:00,FB ADS,PRO:EMEM_OBJ:CONV_CN:NORM_AW:SING_CONT:4STEPS,IT-TECH-270121,23846839370600634,arnu any,EB Company (อุตสาหกรรมผลิตขนมและเบเกอรี่ นครปฐม),0896116463,...,2021-01-30 06:45:41+00:00,,arnu any: EB Company (อุตสาหกรรมผลิตขนมและเบเก...,True,True,5,2021-01-30,13:45:41,Not sure,0
292,329,2021-01-27 09:02:48.441033+00:00,2021-02-22 15:24:23.402881+00:00,GG ADS,Product-250121,โปรแกรม ออนไลน์,e-Memo,ธนัชญา วงศ์ผาบุตร,บริษัท ไอเคพี ซุปเปอร์ เซอร์วิส จำกัด,0960927303,...,2021-01-27 09:02:48+00:00,,ธนัชญา วงศ์ผาบุตร: บริษัท ไอเคพี ซุปเปอร์ เซอ...,True,True,2,2021-01-27,16:02:48,นำคนต่างด้าวมาทำงานในประเทศ,0


In [294]:
mkt_1.isnull().sum()

id                   0
created_at           0
updated_at           0
source               0
compaign_name        0
ads_set_name         0
ads_id               0
name                 0
company_name         0
phone                0
email                8
industry             0
status               0
reason               0
from_url             0
remark               0
product              0
datetime             0
company_size         0
lead_name            0
pipedrive_enable     0
pipedrive_is_send    0
day_of_week          0
date                 0
time                 0
details              8
value                0
dtype: int64

In [300]:
mkt_2 = mkt[(mkt['date'] >= "1/2/2021") & (mkt['date'] <= "28/2/2021")]

In [301]:
mkt_2 = mkt_2[mkt_2['status'] == 'WIN']

In [302]:
mkt_2

,id,created_at,updated_at,source,compaign_name,ads_set_name,ads_id,name,company_name,phone,...,datetime,company_size,lead_name,pipedrive_enable,pipedrive_is_send,day_of_week,date,time,details,value
2,509,2021-02-20 19:24:01.275253+00:00,2021-02-22 15:24:22.812849+00:00,GG ORG,,,,ชุติพนธ์,ชุติพนธ์,0916982333,...,2021-02-20 19:24:00+00:00,,ชุติพนธ์: ชุติพนธ์,True,True,6,2021-02-21,02:24:00,None,0
9,198,2021-01-20 10:28:14.522182+00:00,2021-02-22 15:24:22.905731+00:00,FB ADS,PRO:EMEM_OBJ:CONV_AW:SING_CONT:4STEPS_,BISS-JOBSTITLE-TECH-060121,23846644567020634,Chaowalit Yongkunsiri,บจก. บีเอสแอล โกลบอล,925262591,...,2021-01-10 05:42:00+00:00,,Chaowalit Yongkunsiri: บจก. บีเอสแอล โกลบอล,False,False,6,2021-01-10,12:42:00,,0
10,512,2021-02-22 09:51:36.106114+00:00,2021-02-22 15:24:22.815781+00:00,FB ADS,,,,ฉลองรัฐ,East Water,,...,2021-02-22 09:50:11+00:00,,ฉลองรัฐ: East Water,True,True,0,2021-02-22,16:50:11,None,0
16,234,2021-01-20 10:28:15.357089+00:00,2021-02-22 15:24:22.844946+00:00,MANUAL,,,,duangjai sittikan,เอสพีเค ซาวด์ ซิสเต็มส์,884967295,...,2021-01-18 03:33:00+00:00,,duangjai sittikan: เอสพีเค ซาวด์ ซิสเต็มส์,False,False,0,2021-01-18,10:33:00,,0
18,514,2021-02-23 04:35:46.112752+00:00,2021-02-23 04:35:46.112800+00:00,FB ADS,,,,คุณแก้ว,V Property,081-910-0076,...,2021-02-23 04:32:50+00:00,,คุณแก้ว: V Property,True,True,1,2021-02-23,11:32:50,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,419,2021-02-08 07:51:09.765662+00:00,2021-02-22 15:24:23.400662+00:00,FB ADS,PRO:EMEM_OBJ:CONV_CN:NORM_AW:SING_CONT:4STEPS,Lookalike-call-to-action-website-030221,23846902616260634,คุณ ไกรภพ,DoubleA,085-835-1556,...,2021-02-08 07:51:09+00:00,,คุณ ไกรภพ: DoubleA,True,True,0,2021-02-08,14:51:09,None,0
292,329,2021-01-27 09:02:48.441033+00:00,2021-02-22 15:24:23.402881+00:00,GG ADS,Product-250121,โปรแกรม ออนไลน์,e-Memo,ธนัชญา วงศ์ผาบุตร,บริษัท ไอเคพี ซุปเปอร์ เซอร์วิส จำกัด,0960927303,...,2021-01-27 09:02:48+00:00,,ธนัชญา วงศ์ผาบุตร: บริษัท ไอเคพี ซุปเปอร์ เซอ...,True,True,2,2021-01-27,16:02:48,นำคนต่างด้าวมาทำงานในประเทศ,0
295,401,2021-02-04 00:27:25.390476+00:00,2021-02-22 15:24:23.409146+00:00,FB ADS,PRO:EMEM_OBJ:CONV_CN:NORM_AW:SING_CONT:SING,BISS-JOBSTITLE-TECH-270121,23846847881240634,Kitichai,Boncafe (Thailand) Ltd.,061-636-5944,...,2021-02-04 00:27:25+00:00,,Kitichai: Boncafe (Thailand) Ltd.,True,True,3,2021-02-04,07:27:25,None,0
296,190,2021-01-20 10:28:14.356196+00:00,2021-02-22 15:24:23.411073+00:00,FB POST,,,,Tarit Nakavajara,24 FIX,855667715,...,2021-01-08 04:46:00+00:00,,Tarit Nakavajara: 24 FIX,False,False,4,2021-01-08,11:46:00,,0


In [303]:
ps = pd.read_sql("marketing_prospect", engine)

In [304]:
ps

,id,created_at,updated_at,lead_id,deal_id,person_id,org_id
